In [ ]:
! wget https://raw.githubusercontent.com/yandexdataschool/mlhep2019/master/notebooks/day-5/tracking/metrics.py

In [ ]:
! wget https://raw.githubusercontent.com/yandexdataschool/mlhep2019/master/notebooks/day-5/tracking/user_test_submission.py

In [ ]:
! wget https://raw.githubusercontent.com/yandexdataschool/mlhep2019/master/notebooks/day-5/tracking/public_train.csv

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas
import numpy

import user_test_submission as submission

In [ ]:
import numpy as np
import math
from metrics import predictor
from sklearn.base import BaseEstimator

# Track Pattern Recognition using Linear Approximation of a Track
Original competition: https://ramp.studio/problems/HEP_tracking

## Introduction

Track pattern recognition is an early step of the reconstruction of data coming from a particle detector. It recognizes tracks among the subdetectors hits. Reconstructed track parameters allow to estimate the particle deviation in a magnetic field, and thus reconstruct its charge and momentum. This information is used for the reconstruction of the decay vertex, to identify the mother particle and for further particle identification.

There is wide variety of the track pattern recognition methods. They differ in how they process the hits, what kind of tracks they are able to recognize and which requirements these tracks should satisfy. Therefore, specifics of an experiment and the detector geometry affect the tracking performance and track pattern recognition methods should be adapted to it accordingly.

In this notebook a track pattern recognition for a 2D detector with circular geometry and uniform magnetic field is considered. The detector schema with hits and tracks of an event is shown in the figure below. The challenge is to recognize tracks of an event with the highest efficiecny. It supposed that one hit can belong to only one track. 

<img src="pic/detector.png" /> <br>




## Quick Glossary

We use the following vocabulary in the context of practice session. It may be slightly different from the general high-energy physics context. Some terms are not used in the workbook, but kept here in case they are used in the discussions.

* **event**: a recorded (or simulated) collision in which many particles are present, the basic unit of particle physics data
* **pixel**: the smallest detector unit
* **hit**: a pixel through which a particle has passed and left signal in a given event
* **cluster**: a set of hits, belonging (or predicted to be belonging) to the trajectory of a single particle
* **reconstruct**: same thing as predict, but may also refer to further derived quantities
* **track**: a reconstructed particle, that is, a cluster but also including additional derived information, such as the overall curvature and angle
* **vertex**: the point close to the center of the detector, from which the particles have originated
* **impact parameter**: the shortest distance between a track and the origin
* **(momentum/angle/etc) resolution** : width of the normal distribution of the difference between a predicted quantity and the true value of that quantity

## Objectives

The main objective of the challenge is the optimal matching of the hits in an event. The positions of the hits in the detector are provided as input data, and the user is expected to implement a clustering scheme (trained on data if needed), so that every hit is assigned to a cluster id.

The value of the cluster id itself is not relevant for the task, what is relevant is which hits are clustered together, and whether this clustering corresponds well to the input particles. The score function that describes this is included in the notebook, and details will be mentioned there.

## Application

The user is expected to implement the class `clusterer.py`, which contains the `__init__`, `fit`, and `predict_single_event` functions.

* **`__init__`** is where parameters should be set.
* **`fit`** is the training function (not to be confused with track-fitting), where the algorithm has access to the ground-truth. This function is to be run once on an input array that contains a set of training events. The user is able to implement any event-level or particle-level segmentation of the input array in order to set up the training in any desired way.
* **`predict_single_event`** is the function to reconstruct the hit clusters (tracks), returning an array of predicted (reconstructed) ids associated to each hit in the input array. This function takes only the hits from a single event as input, with the event_id dropped, and the RAMP machinery takes care of running this function on each event.

The **task**  is to implement this class in a way that the predict_single_event function returns a numpy array of assigned cluster ids. At any level of this task, machine-learning techniques can be employed for sub-tasks defined by the user.

## Detector

Image from the Atlas experiment:

<img src="pic/atlas2009.png" /> <br>

The data provided to the user is a list of hit positions from a simple toy detector model that mimics the Atlas detector design (which is generic enough for recent silicon-based tracking detectors). The detector has an onion-like geometry with 9 layers surrounding the origin with polar distances of $R = [39,85,155,213,271,405,562,762,1000]$ cm. These layers have a very small thicknesses compared to the distances, therefore the thickness can be neglected.

Each layer is segmented in azimuth with high granularity. There are ($2\pi$R/pitch)+1 pixels in every layer, where pitch is 0.025 cm for layers 0-4 and 0.05 cm for layers 5-9.

Every "pixel" corresponds to the smallest detector unit defined by `layer` and `iphi` (azimuthal index).

## Simulation

The task uses a toy model for particle generation and simulation, in which a Poisson distribution is sampled to determine the number of particles in each event, with an average of 10 particles per event.

The particles are sampled uniformly in azimuth and momentum with bounds on the momentum. Each particle originates from a vertex that is also randomly sampled from a narrow normal distribution around the origin. The proper dimensions of the momentum and position and determination of these values for the tracks are beyond the scope of the challenge.

The particles generated this way are simulated in a uniform magnetic field. The detector material is considered to cause multiple-scattering, and this is implemented as a random rotation of the particle momentum at every detector position, sampled from a narrow normal distribution that roughly corresponds to the material of the Atlas tracker.

In addition, some hit inefficiency is simulated by a random drop of some hits (with 3% probability), and a particle stopping probability of 1% is applied at each layer to simulate effects of hadronic interactions. Keeping these in mind, the algorithms might be desired to be able to handle cases when the particle doesn't have a hit on every layer.

Since the detector has a very high granularity in azimuth, the cases where two particles pass through a single pixel are neglected (less than 0.2% probability).



## About this notebook

This notebook demonstrate how linear approximation of a track can be used for track pattern recognition. The notebook describes input data, the track pattern recognition method and qualyti metrics, and shows how to use them.

In [ ]:
#!sudo pip install sklearn==0.18.1

# Input data

In [ ]:
data = pandas.read_csv('public_train.csv', index_col=False)

data.head()

## Exploratory data analysis

### Data Preparation

In [ ]:
X = data.drop(['cluster_id'], axis=1).values
y = data[['event_id', 'cluster_id']].values

### Visualize the hits in a single event

In [ ]:
# Get event ids
event_ids = X[:, 0]

# Estimate unique event ids
unique_event_ids = np.unique(event_ids)

# Choose an event
i_event = unique_event_ids[0]

X_event = X[event_ids == i_event]
y_event = y[event_ids == i_event]

In [ ]:
def plot_event(X, y=None, is_plot=True):
    
    cmap = ['k', 'b', 'y', 'g', 'r', 'k']
    R=[39, 85, 155, 213, 271, 405, 562, 762, 1000]
    
    hit_x = X[:, 3]
    hit_y = X[:, 4]
    
    plt.figure(figsize=(6, 6))
    if y is None:
        plt.scatter(hit_x, hit_y)
        
    if y is not None:
        for i_track in np.unique(y):
            track_mask = y == i_track
            track_x = hit_x[track_mask]
            track_y = hit_y[track_mask]
            track_r = np.sqrt(track_x**2 + track_y**2)
            sorted_ids = np.argsort(track_r)
            if is_plot:
                plt.plot(track_x[sorted_ids], track_y[sorted_ids], color=cmap[int(i_track) % len(cmap)])
            plt.scatter(track_x[sorted_ids], track_y[sorted_ids], color=cmap[int(i_track) % len(cmap)])
        
    for ar in R:
        circle=plt.Circle((0,0), ar, color='0.5', fill=False)
        plt.gcf().gca().add_artist(circle)
    
    plt.xlim(-1100, 1100)
    plt.ylim(-1100, 1100)
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.xlabel('X', size=12)
    plt.ylabel('Y', size=12)
    plt.show()

In [ ]:
print("You have this:")
plot_event(X_event, None)

print("You need to get this:")
plot_event(X_event, y_event[:, 1])

## Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split


event_ids = numpy.unique(data['event_id'].values)

event_ids_train, event_ids_test = train_test_split(event_ids, 
                                                   test_size=1000, 
                                                   random_state=42)

X_train, y_train = X[data['event_id'].isin(event_ids_train)], y[data['event_id'].isin(event_ids_train)]
X_test, y_test   = X[data['event_id'].isin(event_ids_test)],  y[data['event_id'].isin(event_ids_test)]

# Track Pattern Recognition

## Clustering based approach

This method is based on the clustering methods. 

In [ ]:
class Clusterer(object):

    def __init__(self, cluster=None):

        self.cluster = cluster

    def get_polar(self, x, y):

        x = numpy.array(x)
        y = numpy.array(y)

        phi = numpy.arctan2(y, x)
        r = numpy.sqrt(x**2 + y**2)

        return r, phi

    
    def fit(self, X, y):
        pass

    
    def predict_single_event(self, X):

        x, y = X[:, 3], X[:, 4]
        r, phi = self.get_polar(x, y)

        self.cluster.fit(phi.reshape(-1, 1))
        labels = self.cluster.labels_
        
        return labels

In [ ]:
from sklearn.cluster import KMeans

ctr = Clusterer(cluster=KMeans(n_clusters=5, n_init=1))

### Test on single event

In [ ]:
y_event_pred = ctr.predict_single_event(X_event)
y_event_pred

In [ ]:
print("Recognized tracks:")
plot_event(X_event, y_event_pred)

### Test on several events

In [ ]:
%%time
from metrics import predictor
y_pred_test = predictor(ctr, X_test, y_test)

In [ ]:
score = submission.score_function(y_test, y_pred_test)
print("Score: {:1.4f}".format(score))

# Task 1

Try to improve the score of track pattern recognition using clustering algorithms.

In [ ]:
### Your code is here ###

score = submission.score_function(y_test, y_pred_test)
print("Score: {:1.4f}".format(score))

# RNN based tracks pattern recognition

Original article: https://www.epj-conferences.org/articles/epjconf/pdf/2017/19/epjconf_ctdw2017_00003.pdf

We will use a basic LSTM model architecture used to classify hits for one track. The LSTM and a fully-connected layer with a softmax activation read the pixel arrays and predict which pixels belong to the target track.

<img src="pic/model.png" width="600"/> <br>

## Data preparation

In [ ]:
num_phi_bin = 100

In [ ]:
# Data preprocessing functions

class det_geo():
    layer_r     = np.array([39,    85,    155,   213,   271,   405,  562,  762,  1000])
    layer_pitch = np.array([0.025, 0.025, 0.025, 0.025, 0.025, 0.05, 0.05, 0.05, 0.05])
    num_layers = layer_r.shape[0]
    max_phi = (2 * np.pi * layer_r / layer_pitch + 1).astype(np.int)

    
def rescale_phi(phi, nbin, layer):
    return (phi * float(nbin) / det_geo.max_phi[layer.astype(np.int)]).astype(np.int)


def event_hits_transform(X, y=None, num_phi_bin=100):
    
    evids, layers, phis = X[:,0].astype(np.int), X[:,1].astype(np.int), X[:,2].astype(np.int)
    phis = rescale_phi(phis, num_phi_bin, layers)
    
    # Count the number of first-layer hits, which will be my seeds
    seed_idx = np.where(layers == 0)[0]
    num_seeds = seed_idx.size

    # Training input will contain the seed hit followed by the rest of the event
    train_input = np.zeros((num_seeds, det_geo.num_layers, num_phi_bin))
    # Training targets will be images of the individual signal tracks
    train_target = np.zeros_like(train_input)

    # Loop over training samples to prepare (seeds)
    for i_sample in range(num_seeds):
        
        i_hit = seed_idx[i_sample]

        # Fill the input first layer with just the seed hit
        train_input[i_sample, 0, phis[i_hit]] = 1
        # Fill the other layers with all remaining event hits
        train_input[i_sample, layers[layers > 0], phis[layers > 0]] = 1

        # Fill target with hits from this track
        if y is not None:
            track_ids = y[:, 1]
            seed_track_id = y[i_hit, 1]
            train_target[i_sample, layers[track_ids == seed_track_id], phis[track_ids == seed_track_id]] = 1
        
    return train_input, train_target

In [ ]:
# Data visualization function

def plot_event_polar(X):
    
    plt.figure(figsize=(6, 6))
    
    plt.pcolor(X.T)
    
    plt.xticks(size=12)
    plt.yticks(size=12)
    
    plt.xlabel('Layer bins', size=12)
    plt.ylabel('Phi bins', size=12)
    
    plt.show()

### Preprocess one event

In [ ]:
# Preprocessing
X_event_polar, y_event_polar = event_hits_transform(X_event, y_event, num_phi_bin)

In [ ]:
# Display shapes
X_event_polar.shape, y_event_polar.shape

In [ ]:
i_train = 0

print("Original event hits:")
plot_event(X_event, None)

print("RNN inputs for one track of the event:")
plot_event_polar(X_event_polar[i_train])

print("RNN outputs for the track:")
plot_event_polar(y_event_polar[i_train])

### Preprocess all training events

In [ ]:
# Preprocessing
train_input = []
train_target = []

event_ids = X_train[:, 0]
for i_event in np.unique(event_ids):
    
    # Take one event
    X_ev = X_train[event_ids == i_event]
    y_ev = y_train[event_ids == i_event]

    # Preprocess this event
    train_input_ev, train_target_ev = event_hits_transform(X_ev, y_ev, num_phi_bin)
    
    # Store the preprocessed events
    train_input.append(train_input_ev)
    train_target.append(train_target_ev)

X_train_polar = np.concatenate(tuple(train_input))
y_train_polar = np.concatenate(tuple(train_target))

In [ ]:
# Display shapes
X_train_polar.shape, y_train_polar.shape

## Define RNN model

In [ ]:
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm_notebook

# Estimate device to train a model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define a model
class Model(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.lstm1 = nn.LSTM(100, 50, batch_first=True)
        self.dense1 = nn.Linear(50, 100)
        self.softmax = nn.Softmax()
 
    def forward(self, x):
        
        hid_sequence, (c_last, h_last) = self.lstm1(x)
        hid_sequence = self.dense1(hid_sequence)
        logits = self.softmax(hid_sequence)
        return logits

## Fit the RNN model

In [ ]:
class MyClassifier(object):
    
    def __init__(self, model, n_epochs=10, batch_size=64):
        """
        Class to fit a pytorch model and to use it to make predictions.
        
        Parameters
        ----------
        model: object
            PyTorch model.
        n_epochs: int
            Number of epochs to fit the model.
        batch_size: int
            The batch size used for the model fitting.
        """
        
        self.model = model
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        
    
    def fit(self, X, y):
        """
        Fit the PyTorch model.
        
        Parameters
        ----------
        X: numpy.array
            The model imput data.
        y: numpy.array
            Targets to predict.
        """
        
        # Convert X and y into torch tensors
        X_tensor = torch.as_tensor(X, dtype=torch.float32, device=device)
        y_tensor = torch.as_tensor(y, dtype=torch.float32, device=device)
        
        # Create dataset for trainig procedure
        train_data = TensorDataset(X_tensor, y_tensor)
        
        # Estimate optimizer
        opt = torch.optim.Adam(self.model.parameters())
        
        # Enable droout
        self.model.train(True)
        
        # Start the model fit
        for epoch_i in range(self.n_epochs):
            loss_history = []
            for x_batch, y_batch in tqdm_notebook(DataLoader(train_data, batch_size=self.batch_size, shuffle=True)):
                # make prediction on a batch
                logits = self.model(x_batch)
                # calculate loss
                loss = -(torch.log(logits) * y_batch).sum(-1).mean()
                # set gradients to zero
                opt.zero_grad()
                # backpropagate gradients
                loss.backward()
                # update the model weights
                opt.step()
                loss_history.append(loss.item())
            print("epoch: %i, mean loss: %.5f" % (epoch_i, np.mean(loss_history)))
    
    def predict(self, X):
        """
        Make predictions for the model.
        
        Parameters
        ----------
        X: numpy.array
            The model imput data.
        """
        
        # Disable droout
        self.model.train(False)
        
        # Convert X and y into torch tensors
        X_tensor = torch.as_tensor(X, dtype=torch.float32, device=device)
        # Make predictions for X 
        y_pred = model(X_tensor)
        
        return y_pred.cpu().detach().numpy()

In [ ]:
# Create a pytorch model
model=Model().to(device=device)

# Fit the model
clf = MyClassifier(model=model, n_epochs=50, batch_size=64)
clf.fit(X_train_polar, y_train_polar)

In [ ]:
# Make predictions for one event
y_event_polar_pred = clf.predict(X_event_polar)

In [ ]:
print("RNN output for the track:")
plot_event_polar(y_event_polar_pred[i_train])

print("True output for the track:")
plot_event_polar(y_event_polar[i_train])

## Test track pattern recognition algorithm

In [ ]:
class Clusterer(object):

    def __init__(self, classifier=None):

        self.classifier = classifier
    
    def fit(self, X, y):
        pass

    
    def predict_single_event(self, X):

        trkid = np.zeros(X.shape[0])
        for i in range(trkid.shape[0]):
            
            lays, phis = X[:,1].astype(np.int), X[:,2].astype(np.int)
            phis = rescale_phi(phis, num_phi_bin, lays)
            lay, phi = lays[i], phis[i]
            
            # Preprocess the event
            X_polar, _ = event_hits_transform(X, None, num_phi_bin)
            # Get predictions for the event
            y_polar_pred = clf.predict(X_polar)

            # Get the list of track scores
            scores = y_polar_pred[:, lays[i], phis[i]]

            # Select the best score
            best = np.argmax(scores)
            trkid[i] = best
            
        return trkid

In [ ]:
ctr = Clusterer(classifier=clf)

y_event_pred = ctr.predict_single_event(X_event)
y_event_pred

In [ ]:
print("Recognized tracks:")
plot_event(X_event, y_event_pred, is_plot=True)

In [ ]:
%%time
from metrics import predictor
y_pred_test = predictor(ctr, X_test, y_test)

In [ ]:
score = submission.score_function(y_test, y_pred_test)
print("Score: {:1.4f}".format(score))

# Task 2

Create a deep model with the following layers:
* Input layer
* LSTM with 200 hidden neurons
* 30% dropout
* LSTM with 150 hidden neurons
* 20% dropout
* Output dense layer with 100 neurons and Softmax activation.

In [ ]:
# Define a model
class Model(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        ### Your code is here ###
 
    def forward(self, x):
        
        ### Your code is here ###
        
        return logits

In [ ]:
# Create a pytorch model
### Your code is here ###

# Fit the model
### Your code is here ###

In [ ]:
%%time
# Calculate the quality score
### Your code is here ###

# Task 3 (for the Jedi)

Try to improve the score. Fill free to change anything you want. 

In [ ]:
### Your code is here ###